In [5]:
!pip install --quiet ultralytics gradio pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00


In [6]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import sqlite3
import os
import datetime

# PHASE 1: Configuration and Setup

MODEL_FILE = "yolov8n.pt"  # YOLO model file
DB_FILE = "product_db.sqlite"  # SQLite database

# Initialize YOLO model
print("Loading YOLOv8n model...")
if not os.path.exists(MODEL_FILE):
    print(f"Downloading YOLOv8n model to {MODEL_FILE}...")
    model = YOLO("yolov8n.pt")
else:
    model = YOLO(MODEL_FILE)
print(" Model loaded successfully!")


# PHASE 2: Database Setup and Management


def ensure_database():

    conn = sqlite3.connect(DB_FILE)
    cur = conn.cursor()

    # Products table
    cur.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            label TEXT UNIQUE,
            name TEXT,
            category TEXT,
            sku TEXT,
            brand TEXT
        )
    ''')

    # Detection history table
    cur.execute('''
        CREATE TABLE IF NOT EXISTS detection_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            label TEXT,
            name TEXT,
            confidence REAL,
            status TEXT,
            image_name TEXT
        )
    ''')

    conn.commit()
    conn.close()
    print("Database initialized!")

# Initialize database
ensure_database()


# PHASE 3: Database Helper Functions

def add_product_to_db(label, name, category, sku, brand):

    label = label.strip().lower()
    name = name.strip()

    if not label or not name:
        return " Error: Label and name are required!"

    try:
        conn = sqlite3.connect(DB_FILE)
        cur = conn.cursor()
        cur.execute('''
            INSERT OR REPLACE INTO products (label, name, category, sku, brand)
            VALUES (?, ?, ?, ?, ?)
        ''', (label, name, category.strip(), sku.strip(), brand.strip()))
        conn.commit()
        conn.close()
        return f" Product saved: {name} (label={label})"
    except Exception as e:
        return f" Error: {str(e)}"

def view_all_products():
    """View all products in database"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cur = conn.cursor()
        cur.execute('SELECT label, name, category, sku, brand FROM products ORDER BY label')
        rows = cur.fetchall()
        conn.close()

        if not rows:
            return "No products in database."

        output = " **Products in Database:**\n\n"
        for row in rows:
            output += f"**{row[1]}**\n"
            output += f"  - Label: `{row[0]}`\n"
            output += f"  - Category: {row[2]}\n"
            output += f"  - SKU: {row[3]}\n"
            output += f"  - Brand: {row[4]}\n\n"

        return output
    except Exception as e:
        return f" Error: {str(e)}"

def delete_product_from_db(label):
    """Delete a product by label"""
    label = label.strip().lower()

    if not label:
        return " Error: Please enter a label to delete!"

    try:
        conn = sqlite3.connect(DB_FILE)
        cur = conn.cursor()
        cur.execute('DELETE FROM products WHERE label = ?', (label,))
        deleted_count = cur.rowcount
        conn.commit()
        conn.close()

        if deleted_count > 0:
            return f"Deleted product with label='{label}'"
        else:
            return f" No product found with label='{label}'"
    except Exception as e:
        return f" Error: {str(e)}"

def log_detection(timestamp, label, name, confidence, status):
    """Log detection to database"""
    try:
        conn = sqlite3.connect(DB_FILE)
        cur = conn.cursor()
        cur.execute('''
            INSERT INTO detection_log (timestamp, label, name, confidence, status, image_name)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (timestamp, label, name, confidence, status, 'uploaded_image'))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Error logging detection: {e}")

def view_detection_history(limit=50):

    try:
        conn = sqlite3.connect(DB_FILE)
        cur = conn.cursor()
        cur.execute('''
            SELECT timestamp, label, name, confidence, status
            FROM detection_log
            ORDER BY id DESC
            LIMIT ?
        ''', (limit,))
        rows = cur.fetchall()
        conn.close()

        if not rows:
            return " No detection history yet."

        output = " **Detection History (Recent):**\n\n"
        for row in rows:
            status_icon = "" if row[4] == "Verified" else "" if row[4] == "Review" else ""
            output += f"{status_icon} **{row[2]}** (label: `{row[1]}`)\n"
            output += f"  - Confidence: {row[3]:.2%}\n"
            output += f"  - Status: {row[4]}\n"
            output += f"  - Time: {row[0]}\n\n"

        return output
    except Exception as e:
        return f" Error: {str(e)}"

# PHASE 4-6: YOLO Detection and Identification


def detect_and_identify(image, conf_threshold=0.4):


    if image is None:
        return None, "Please upload an image!"

    try:
        # Convert to PIL Image if needed
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)

        # Resize for better performance
        image.thumbnail((800, 800), Image.LANCZOS)

        # PHASE 4: Run YOLO detection
        print("🔍 Running YOLO detection...")
        results = model.predict(np.array(image), conf=conf_threshold, verbose=False)
        boxes = results[0].boxes

        if len(boxes) == 0:
            return image, "**No products detected**\n\nTry:\n- Better lighting\n- Different angle\n- Remove background clutter"

        # PHASE 6: Draw bounding boxes
        draw = ImageDraw.Draw(image)

        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        detected_products = []

        # PHASE 5: Process each detection
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls].lower()

            # Choose color based on confidence
            color = "lime" if conf >= 0.70 else "orange"

            # Draw bounding box
            draw.rectangle([x1, y1, x2, y2], outline=color, width=3)

            # Draw label
            label_text = f"{label} {conf:.2f}"
            draw.rectangle([x1, max(y1-25, 0), x1+150, y1], fill=color)
            draw.text((x1+5, max(y1-22, 2)), label_text, fill="white", font=font)

            # PHASE 5: Lookup in database
            conn = sqlite3.connect(DB_FILE)
            cur = conn.cursor()
            cur.execute('SELECT name, category, sku, brand FROM products WHERE label = ?', (label,))
            row = cur.fetchone()
            conn.close()

            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            if row:
                name, category, sku, brand = row
                status = "Verified" if conf >= 0.70 else "Review"

                # Log detection
                log_detection(timestamp, label, name, conf, status)

                detected_products.append({
                    'label': label,
                    'name': name,
                    'category': category,
                    'sku': sku,
                    'brand': brand,
                    'confidence': conf,
                    'status': status
                })
            else:
                name = "Unknown Product"
                status = "Unknown" if conf >= 0.70 else "Review"

                # Log unknown detection
                log_detection(timestamp, label, name, conf, status)

                detected_products.append({
                    'label': label,
                    'name': name,
                    'category': 'N/A',
                    'sku': 'N/A',
                    'brand': 'N/A',
                    'confidence': conf,
                    'status': status
                })

        # Format results
        result_text = f"🎯 **Detected {len(detected_products)} Product(s)**\n\n"

        for i, prod in enumerate(detected_products, 1):
            status_icon = "" if prod['status'] == "Verified" else "" if prod['status'] == "Review" else ""

            result_text += f"{status_icon} **{i}. {prod['name']}**\n"
            result_text += f"   - **Detected As:** `{prod['label']}`\n"
            result_text += f"   - **Confidence:** {prod['confidence']:.2%}\n"
            result_text += f"   - **Category:** {prod['category']}\n"
            result_text += f"   - **SKU:** {prod['sku']}\n"
            result_text += f"   - **Brand:** {prod['brand']}\n"
            result_text += f"   - **Status:** {prod['status']}\n\n"

        if any(p['status'] == 'Unknown' for p in detected_products):
            result_text += "\n💡 **Tip:** Unknown products can be added via the Admin tab.\n"

        print(f"Detection complete: {len(detected_products)} products found")
        return image, result_text

    except Exception as e:
        error_msg = f" **Error during detection:** {str(e)}\n\nPlease try again."
        print(f"Error: {e}")
        return image, error_msg

# PHASE 7: Gradio Interface


def create_interface():

    with gr.Blocks(title="Product Image Recognition System", theme=gr.themes.Soft()) as demo:

        gr.Markdown("""
        #  Product Image Recognition System
        ### Powered by YOLOv8n + SQLite Database

        Upload product images to automatically detect, identify, and catalog items using AI.
        """)

        with gr.Tabs():

            # TAB 1: Detection
            with gr.Tab("📸 Detect & Identify"):
                gr.Markdown("### Upload a product image for AI-powered recognition")

                with gr.Row():
                    with gr.Column(scale=1):
                        input_image = gr.Image(type="pil", label="Upload Product Image")
                        conf_slider = gr.Slider(0.1, 0.9, value=0.4, step=0.05,
                                               label="Confidence Threshold",
                                               info="Minimum confidence for detection")
                        detect_btn = gr.Button(" Detect & Identify", variant="primary", size="lg")

                    with gr.Column(scale=1):
                        output_image = gr.Image(type="pil", label="Annotated Result")
                        output_text = gr.Markdown(label="Detection Results")

                detect_btn.click(
                    fn=detect_and_identify,
                    inputs=[input_image, conf_slider],
                    outputs=[output_image, output_text]
                )

                gr.Markdown("""
                ###  How it works:
                1. **Upload** a product image (JPG, PNG, etc.)
                2. **Adjust** confidence threshold if needed
                3. **Click** "Detect & Identify" button
                4. View **annotated image** with bounding boxes
                5. See **product details** from database
                """)

            # TAB 2: Admin
            with gr.Tab("🛠️ Admin - Manage Products"):
                gr.Markdown("### Add, update, or delete products in the database")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### Add/Update Product")
                        label_input = gr.Textbox(label="YOLO Label (Required)",
                                                placeholder="e.g., bottle, laptop, chair",
                                                info="Must match YOLO class names (case-insensitive)")
                        name_input = gr.Textbox(label="Product Name (Required)",
                                               placeholder="e.g., Stainless Steel Water Bottle")
                        category_input = gr.Textbox(label="Category",
                                                   placeholder="e.g., Home & Kitchen")
                        sku_input = gr.Textbox(label="SKU",
                                             placeholder="e.g., SSWB-2024-002")
                        brand_input = gr.Textbox(label="Brand",
                                               placeholder="e.g., HydroLife")

                        add_btn = gr.Button("Add/Update Product", variant="primary")
                        add_status = gr.Textbox(label="Status", interactive=False)

                        add_btn.click(
                            fn=add_product_to_db,
                            inputs=[label_input, name_input, category_input, sku_input, brand_input],
                            outputs=add_status
                        )

                    with gr.Column():
                        gr.Markdown("####  View All Products")
                        view_btn = gr.Button("View Products Database", variant="secondary")
                        products_display = gr.Markdown(label="Products")

                        view_btn.click(
                            fn=view_all_products,
                            inputs=None,
                            outputs=products_display
                        )

                        gr.Markdown("---")
                        gr.Markdown("####  Delete Product")
                        delete_label = gr.Textbox(label="Label to Delete",
                                                 placeholder="e.g., bottle")
                        delete_btn = gr.Button(" Delete Product", variant="stop")
                        delete_status = gr.Textbox(label="Status", interactive=False)

                        delete_btn.click(
                            fn=delete_product_from_db,
                            inputs=delete_label,
                            outputs=delete_status
                        )

            # TAB 3: History
            with gr.Tab(" Detection History"):
                gr.Markdown("### View recent detection logs")

                history_btn = gr.Button(" Refresh History", variant="secondary")
                history_display = gr.Markdown(label="History")

                history_btn.click(
                    fn=view_detection_history,
                    inputs=None,
                    outputs=history_display
                )

                # Auto-load on tab open
                demo.load(fn=view_detection_history, inputs=None, outputs=history_display)

        gr.Markdown("""
        ---
        ###  Notes:
        - **Labels** must match YOLO class names (80+ classes available)
        - **Verified:** Confidence ≥ 70% | **Review:** Confidence < 70%
        - Detection history stores last 100 detections
        - Database: `product_db.sqlite` (SQLite)
        """)

    return demo

# MAIN: Launch Application


if __name__ == "__main__":
    print("\n" + "="*60)
    print(" Starting Product Image Recognition System")
    print("="*60)

    # Add some sample products if database is empty
    conn = sqlite3.connect(DB_FILE)
    cur = conn.cursor()
    cur.execute('SELECT COUNT(*) FROM products')
    count = cur.fetchone()[0]
    conn.close()

    if count == 0:
        print("\n Adding sample products to database...")
        sample_products = [
            ("bottle", "Stainless Steel Water Bottle", "Home & Kitchen", "SSWB-2024-002", "HydroLife"),
            ("laptop", "Apple MacBook Pro", "Electronics", "MBP-2024-006", "Apple"),
            ("cell phone", "iPhone 15 Pro", "Electronics", "IP15-2024-001", "Apple"),
            ("backpack", "Leather Backpack", "Accessories", "LBP-2024-005", "UrbanCarry"),
            ("chair", "Office Chair Executive", "Furniture", "OCE-2024-007", "ComfortSeating"),
            ("clock", "Smart Fitness Watch", "Electronics", "SFW-2024-004", "FitTrack"),
            ("cup", "Ceramic Coffee Mug", "Home & Kitchen", "CCM-2024-008", "MugLife"),
            ("book", "Python Programming Guide", "Books", "PPG-2024-009", "TechBooks"),
            ("keyboard", "Mechanical Keyboard RGB", "Electronics", "MKR-2024-010", "KeyTech"),
            ("mouse", "Wireless Gaming Mouse", "Electronics", "WGM-2024-011", "MouseCo"),
        ]

        for product in sample_products:
            add_product_to_db(*product)

        print(f" Added {len(sample_products)} sample products!")

    # Create and launch interface
    demo = create_interface()

    print("\n Launching Gradio interface...")
    print("="*60)

    # Launch with share=True for public link
    demo.launch(
    share=False,
    server_name="0.0.0.0",
    server_port=None,
    show_error=True
)


    demo.launch(share=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading YOLOv8n model...
 Model loaded successfully!
Database initialized!

 Starting Product Image Recognition System

 Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1ea63f3b9e64ec295.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
